<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-27 12:59:58
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.87 L
Current:  1.29 C
-------------------
Today PnL: 44.52 K (0.35%)
Current PnL: -16.37 L (-11.61%)
CY Booked + Current PnL: -12.07 L (-8.56%)
-------------------
Total profit:  3.82 L
Total loss:  -20.19 L
-------------------
Total Booked + Current PnL: 14.91 L (13.07%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.32%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.09 C
Est FTT PnL: 79.35 L (61.33%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.41 C (108.71%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,2.11,H-LC,48.94,101114.0,17311.0,6684.0,0.22,20.66,6.61,28.63,42.0,2.59,0.79,35.12,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,4.23,X-LC,47.87,154856.0,14047.0,11475.0,-0.98,9.98,7.41,18.13,17.0,1.22,1.21,22.27,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,89.36,99471.0,21109.0,12921.0,0.06,26.94,12.99,43.43,71.0,1.63,0.78,33.04,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,8.32,H-SC,88.30,159698.0,21838.0,15922.0,0.05,15.84,9.97,27.39,118.0,1.37,1.25,38.41,X40,ATH,FMCG
56,PIDILITIND,2504.06,3576.00,-18.11,X-LC,22.34,90765.0,15643.0,16519.0,-0.65,20.82,18.20,42.81,11.0,0.95,0.71,13.83,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,AWL,342.88,485.00,-68.17,H-MC,8.51,172752.0,-44977.0,135230.0,5.14,-20.66,78.28,41.45,86.0,-0.33,1.35,15.40,XY24,NTT,FMCG
88,VALIANTORG,512.64,838.00,-354.52,H-SC,97.87,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,7.39,H-SC,100.00,124023.0,26376.0,21295.0,4.05,27.01,17.17,48.82,138.0,1.24,0.97,37.78,X40,ATH,PHARMA
60,RBLBANK,230.61,388.00,37.52,M-SC,38.30,213919.0,-20381.0,180291.0,2.90,-8.70,84.28,68.25,216.0,-0.11,1.68,40.87,XY24,NTT,BANKS
78,TANLA,917.30,1963.11,-37.40,H-SC,98.94,153233.0,-86182.0,359132.0,2.76,-36.00,234.37,114.01,110.0,-0.24,1.20,40.54,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.0,65.29,X-SC,82.98,110314.0,-41677.0,115797.0,-2.33,-27.42,104.97,48.77,59.0,-0.36,0.87,26.71,XR,NTT,JEWELLERY
80,TATAMOTORS,771.32,1065.0,-52.64,M-LC,64.89,210901.0,-15096.0,101148.0,-1.26,-6.68,47.96,38.07,68.0,-0.15,1.65,24.16,XY24,NTT,AUTO
3,ACC,2579.05,3906.0,-34.60,M-MC,25.53,141255.0,-47016.0,143882.0,-1.19,-24.97,101.86,51.45,153.0,-0.33,1.11,6.97,XY24,BTT,CEMENT
81,TCS,3841.06,4998.0,-16.16,X-LC,37.23,241493.0,-23540.0,103359.0,-1.06,-8.88,42.80,30.12,2.0,-0.23,1.89,8.31,X40,BTT,IT
41,INFY,1450.31,2275.0,-12.78,X-LC,55.32,248676.0,18077.0,113048.0,-1.04,7.84,45.46,56.86,1.0,0.16,1.95,13.50,X40,BTT,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.78,X-LC,55.32,248676.0,18077.0,113048.0,-1.04,7.84,45.46,56.86,1.0,0.16,1.95,13.50,X40,BTT,IT
81,TCS,3841.06,4998.00,-16.16,X-LC,37.23,241493.0,-23540.0,103359.0,-1.06,-8.88,42.80,30.12,2.0,-0.23,1.89,8.31,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.11,X-LC,86.17,182700.0,-17610.0,76405.0,-0.72,-8.79,41.82,29.35,4.0,-0.23,1.43,26.22,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-9.47,X-LC,80.85,81835.0,-28337.0,117409.0,0.76,-25.72,143.47,80.85,5.0,-0.24,0.64,20.53,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-5.19,X-LC,11.70,213493.0,-3246.0,67912.0,0.15,-1.50,31.81,29.84,6.0,-0.05,1.67,11.99,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,7.39,H-SC,100.00,124023.0,26376.0,21295.0,4.05,27.01,17.17,48.82,138.0,1.24,0.97,37.78,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,89.36,99471.0,21109.0,12921.0,0.06,26.94,12.99,43.43,71.0,1.63,0.78,33.04,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,2.11,H-LC,48.94,101114.0,17311.0,6684.0,0.22,20.66,6.61,28.63,42.0,2.59,0.79,35.12,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-18.11,X-LC,22.34,90765.0,15643.0,16519.0,-0.65,20.82,18.20,42.81,11.0,0.95,0.71,13.83,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.00,M-SC,69.15,145065.0,7487.0,86923.0,-0.29,5.44,59.92,68.62,182.0,0.09,1.14,25.76,X40N,NTT,TEXTILES
28,GILLETTE,8109.44,10330.69,8.32,H-SC,88.30,159698.0,21838.0,15922.0,0.05,15.84,9.97,27.39,118.0,1.37,1.25,38.41,X40,ATH,FMCG
55,PFIZER,4245.50,6318.30,7.39,H-SC,100.00,124023.0,26376.0,21295.0,4.05,27.01,17.17,48.82,138.0,1.24,0.97,37.78,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,6.73,H-MC,59.57,101320.0,16219.0,26880.0,0.18,19.06,26.53,50.65,92.0,0.60,0.79,27.36,X40N,ATH,CABLES
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,77.66,151854.0,941.0,59512.0,-0.48,0.62,39.19,40.06,89.0,0.02,1.19,24.21,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.89,H-SC,27.66,111328.0,9324.0,35703.0,-0.04,9.14,32.07,44.14,111.0,0.26,0.87,26.87,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,17.02,133066.0,3914.0,83153.0,-0.32,3.03,62.49,67.42,155.0,0.05,1.04,29.42,XR,NTT,BANKS
93,WIPRO,243.46,420.00,-9.71,M-LC,46.81,154740.0,3795.0,105656.0,-0.26,2.51,68.28,72.51,69.0,0.04,1.21,16.51,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,77.66,151854.0,941.0,59512.0,-0.48,0.62,39.19,40.06,89.0,0.02,1.19,24.21,AR,ATH,AUTO
38,INDIAMART,2327.09,4911.36,-52.96,H-SC,42.55,122377.0,-959.0,137931.0,-0.99,-0.78,112.71,111.05,108.0,-0.01,0.96,19.91,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,58.51,62784.0,-50765.0,90817.0,2.64,-44.71,144.65,35.27,238.0,-0.56,0.49,53.67,XR,NTT,HOTELS
83,TRIDENT,37.35,48.00,68.21,M-SC,93.62,80769.0,-11486.0,37792.0,0.21,-12.45,46.79,28.51,220.0,-0.30,0.63,38.21,XR,NTT,TEXTILES
52,MASFIN,326.60,399.50,-22.76,H-SC,62.77,87735.0,-10245.0,32111.0,0.14,-10.46,36.60,22.32,130.0,-0.32,0.69,27.25,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,89.36,99471.0,21109.0,12921.0,0.06,26.94,12.99,43.43,71.0,1.63,0.78,33.04,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,2.11,H-LC,48.94,101114.0,17311.0,6684.0,0.22,20.66,6.61,28.63,42.0,2.59,0.79,35.12,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,7.39,H-SC,100.00,124023.0,26376.0,21295.0,4.05,27.01,17.17,48.82,138.0,1.24,0.97,37.78,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-37.40,H-SC,98.94,153233.0,-86182.0,359132.0,2.76,-36.00,234.37,114.01,110.0,-0.24,1.20,40.54,AR,ATH,IT
88,VALIANTORG,512.64,838.00,-354.52,H-SC,97.87,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,5938.89,M-SC,96.81,66231.0,-27579.0,104201.0,0.26,-29.40,157.33,81.67,203.0,-0.26,0.52,28.26,XR,NTT,CERAMICS
18,CERA,8421.60,10420.45,-18.14,X-SC,95.74,84624.0,-24857.0,50842.0,0.12,-22.70,60.08,23.73,64.0,-0.49,0.66,28.08,X40N,ATH,CERAMICS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.33
1,25,40.29
2,50,69.32


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.27
XY25    14.31
X40     14.21
X40N    13.68
AR      12.55
XR      10.37
X200     1.43
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.43
M-SC    18.06
X-LC    17.31
H-MC     8.75
M-LC     7.74
H-LC     7.27
X-MC     5.65
M-MC     5.61
X-SC     2.86
L-SC     2.31
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.17,0.03,31.34
IT,9.95,-17.96,82.44
BANKS,9.66,-13.61,65.85
MISC,6.51,-30.14,87.71
INSURANCE,6.03,2.27,26.87
FINANCE,5.96,-25.82,85.56
PAINTS,5.46,-18.41,53.45
ELECTRICAL,4.82,-4.34,56.28
AUTO,4.49,-9.06,56.71


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2858721.0,26
AR,1244850.0,13
X40N,1202900.0,17
XR,1126893.0,14
X40,747002.0,11
XY25,477231.0,10
SR,200695.0,2
X200,76405.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2285112.0,21
M-SC,2011747.0,21
X-LC,768748.0,12
M-MC,552103.0,5
H-MC,511119.0,8
X-SC,343114.0,4
M-LC,317514.0,6
X-MC,313910.0,5
L-SC,303344.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922155.0      6
M-SC       XY24      741939.0      6
H-SC       AR        557398.0      3
           X40N      503380.0      6
M-SC       X40N      462906.0      6
X-LC       X40       430962.0      5
M-MC       XY24      388512.0      3
M-SC       XR        290709.0      4
H-SC       XR        264962.0      4
H-LC       AR        248806.0      4
M-SC       X40       215580.0      2
M-LC       XY24      194662.0      4
H-MC       XY24      176573.0      2
L-SC       XR        155099.0      2
X-LC       AR        141583.0      2
X-MC       XY24      140471.0      2
M-SC       XY25      122337.0      1
X-LC       XY25      119798.0      4
X-SC       XR        115797.0      1
           SR        115499.0      1
           X40N      111818.0      2
H-MC       XR        111517.0      1
M-LC       XR        105656.0      1
X-MC       X40N       97916.0      2
M-SC       AR         93080.0      1
           SR         85196.0      1
L-SC       XY24       84212.0      1
M-MC       XR         83153.0      1
           AR         80438.0      1
X-LC       X200       76405.0      1
X-MC       XY25       75523.0      1
H-MC       XY25       73394.0      1
L-SC       AR         64033.0      1
H-MC       X40        63243.0      2
           AR         59512.0      1
L-LC       XY25       40976.0      1
H-SC       X40        37217.0      2
H-LC       XY25       28007.0      1
H-MC       X40N       26880.0      1
M-LC       XY25       17196.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
